# TSA Chapter 9: Prophet vs TBATS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch9_prophet_vs_tbats/TSA_ch9_prophet_vs_tbats.ipynb)

This notebook generates the forecast comparison visualization of Prophet vs TBATS against actual values.

In [ ]:
!pip install numpy pandas matplotlib -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme
BLUE    = '#1A3A6E'
RED     = '#DC3545'
GREEN   = '#2E7D32'
ORANGE  = '#E67E22'
GRAY    = '#666666'
PURPLE  = '#8E44AD'

# Transparent backgrounds
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True

# No top/right spines
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# General styling
plt.rcParams['axes.grid'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['legend.facecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0
plt.rcParams['axes.linewidth'] = 0.6
plt.rcParams['lines.linewidth'] = 1.0


def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')


def legend_outside(ax, ncol=3):
    """Place legend outside bottom of plot."""
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch9_prophet_vs_tbats
# Forecast comparison: Prophet vs TBATS vs Actual
np.random.seed(99)
n_total = 365 + 90  # 1 year train + 90 days forecast
t = np.arange(n_total)
dates_pv = pd.date_range('2022-01-01', periods=n_total, freq='D')

trend_pv = 100 + 0.03 * t
weekly_pv = 8 * np.sin(2 * np.pi * t / 7)
yearly_pv = 15 * np.sin(2 * np.pi * t / 365)
noise_pv = np.random.normal(0, 4, n_total)
actual = trend_pv + weekly_pv + yearly_pv + noise_pv

n_train = 365
train_dates = dates_pv[:n_train]
fc_dates = dates_pv[n_train:]
actual_fc = actual[n_train:]

# Simulated Prophet forecast (slightly biased low)
prophet_fc = (trend_pv[n_train:] + weekly_pv[n_train:] + yearly_pv[n_train:]
              + np.random.normal(-0.5, 2, 90))
prophet_lower = prophet_fc - 1.96 * np.linspace(4, 8, 90)
prophet_upper = prophet_fc + 1.96 * np.linspace(4, 8, 90)

# Simulated TBATS forecast (slightly biased high)
tbats_fc = (trend_pv[n_train:] + weekly_pv[n_train:] + yearly_pv[n_train:]
            + np.random.normal(0.3, 2.5, 90))
tbats_lower = tbats_fc - 1.96 * np.linspace(5, 9, 90)
tbats_upper = tbats_fc + 1.96 * np.linspace(5, 9, 90)

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Top: Prophet
axes[0].plot(train_dates, actual[:n_train], color=GRAY, linewidth=0.5, alpha=0.5,
             label='Historical')
axes[0].plot(fc_dates, actual_fc, color=BLUE, linewidth=1, label='Actual')
axes[0].plot(fc_dates, prophet_fc, color=RED, linewidth=1.5, label='Prophet')
axes[0].fill_between(fc_dates, prophet_lower, prophet_upper, color=RED, alpha=0.15,
                     label='IC 95% Prophet')
axes[0].axvline(x=dates_pv[n_train], color='black', linestyle='--', alpha=0.4)
axes[0].set_title('Prophet Forecast', fontweight='bold')
axes[0].set_ylabel('Value')
legend_outside(axes[0], ncol=4)

# Bottom: TBATS
axes[1].plot(train_dates, actual[:n_train], color=GRAY, linewidth=0.5, alpha=0.5,
             label='Historical')
axes[1].plot(fc_dates, actual_fc, color=BLUE, linewidth=1, label='Actual')
axes[1].plot(fc_dates, tbats_fc, color=GREEN, linewidth=1.5, label='TBATS')
axes[1].fill_between(fc_dates, tbats_lower, tbats_upper, color=GREEN, alpha=0.15,
                     label='IC 95% TBATS')
axes[1].axvline(x=dates_pv[n_train], color='black', linestyle='--', alpha=0.4)
axes[1].set_title('TBATS Forecast', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Value')
legend_outside(axes[1], ncol=4)

plt.tight_layout(h_pad=3.5)
save_chart(fig, 'ch9_prophet_vs_tbats')
plt.show()